In [29]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
## Getting available workspace
ws = Workspace.from_config()

ws.write_config(path='.azureml/')

#experiment_name = 'udacity_project_solution_real'

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127538
Azure region: southcentralus
Subscription id: 30d182b7-c8c4-421c-8fa0-d3037ecfe6d2
Resource group: aml-quickstarts-127538


In [30]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "cpu-cluster"
'''
try:
    # try to get existing compute
    compute_target = ComputeTarget(workspace=ws, name= cluster_name)
    print("Compute exists")
except ComputeTargetExcpetion:
    print("Compute does not exist. Creating...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
'''
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes=20)
    
#check status of the current cluster
print(compute_target.get_status().serialize())


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2020-11-23T15:20:34.622000+00:00', 'errors': None, 'creationTime': '2020-11-23T12:32:30.512193+00:00', 'modifiedTime': '2020-11-23T12:32:48.210959+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [31]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.policy import MedianStoppingPolicy
from azureml.train.hyperdrive import uniform, choice

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###

# Logistic Regression needs 2 parameters - 
# C = Regularization Strength (float)
# max_iter = Max iterations (int)
# We use Random Parameter Sampling initially as it supports discrete as well as continuous parameters
# We will try Bayesian Sampling later
# Check initial values for parameters. Current values are used for trial

ps = RandomParameterSampling({
    "C": uniform(1,5),
    "max_iter": choice(range(1, 100))
})

primary_metric_name = "Accuracy"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

# Specify early termination policy
#policy = ### YOUR CODE HERE ###
policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
est = SKLearn(source_directory = '.',
              entry_script = 'train.py',
              compute_target = compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name=primary_metric_name,
                                     primary_metric_goal=primary_metric_goal,
                                     estimator=est,
                                     #max_total_runs=100)
                                     max_total_runs=1)

In [32]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == 'Completed')

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_7ac47a61-638a-4e29-83d2-bea3bed86700
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_7ac47a61-638a-4e29-83d2-bea3bed86700?wsid=/subscriptions/30d182b7-c8c4-421c-8fa0-d3037ecfe6d2/resourcegroups/aml-quickstarts-127538/workspaces/quick-starts-ws-127538

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-23T15:21:06.187452][API][INFO]Experiment created<END>\n""<START>[2020-11-23T15:21:06.967884][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space<END>\n""<START>[2020-11-23T15:21:07.287329][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()
hyperdrive_best_run_mertics = hyperdrive_best_run.get_metrics()
parameter_values = hyperdrive_best_run.get_details()['runDefinition']['Arguments']
print('best run id : ', hyperdrive_best_run.id)
print('best run accuracy :', hyperdrive_best_run_metrics['Accuracy'])
print('C : ', parameter_values[3])
print('max_iter : ', parameter_values[5])

# Save best model
best_run_model_name = 'Udacity-hyperdrive-model'
best_run_model_path = 'output/model.pkl'
model = hyperdrive_best_run.register_model(model_name=best_run_model_name,
                               model_path=best_run_model_path)


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = Dataset.Tabular.from_delimited_files(path = [(datastore, "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")])
ds.to_pandas_dataframe()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_experiment = Experiment(ws,'automl_test_experiment')
automl_run = automl_experiment.submit(config=automl_config,
                                     sjow_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_run = automl_run.get_best_run_by_primary_metric()
automl_best_run_mertics = automl_best_run.get_metrics()
#parameter_values = automl_best_run.get_details()['runDefinition']['Arguments']
print('best run id : ', automl_best_run.id)
print('best run accuracy :', automl_best_run_metrics['Accuracy'])
'''
print('C : ', parameter_values[3])
print('max_iter : ', parameter_values[5])
'''
# Save best model
best_run_model_name = 'Udacity-automl-model'
best_run_model_path = 'output/model'
model = automl_best_run.register_model(model_name=best_run_model_name,
                               model_path=best_run_model_path)